In [2]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from parse_logs.parse_base import LogDir
from parse_logs.parse_coordinator_logs import read_design

from parse_logs.parse_coordinator_logs import get_flat_design
from flatten_json import flatten

from verification.verify_experiment import check_experiment

sns.set(rc={"figure.figsize":(16, 8)})
# parser.root_data_path = '../tests/data'
#trial_run_objects = parser.get_trial_runs(
#    exp_run_code='experiment_2021_04_01_16_20_00_run_1')

LogDir.base_data_path = '../data'
check_mc, check_te = check_experiment('experiment_2021_07_29_15_33_17_run_1')



failure on parsing the line "14.95, [DEBUG], turtlebot3, {"param_list": [], "skill": "init_robobt", "skill-life-cycle": "SUCCESS", "label": "init_robobt"}
"
'turtlebot3'
failure on parsing the line "16.85, [DEBUG], turtlebot6, {"param_list": [], "skill": "init_robobt", "skill-life-cycle": "SUCCESS", "label": "init_robobt"}
"
'turtlebot6'
failure on parsing the line "17.12, [DEBUG], turtlebot1, {"param_list": [], "skill": "init_robobt", "skill-life-cycle": "SUCCESS", "label": "init_robobt"}
"
'turtlebot1'
failure on parsing the line "18.13, [DEBUG], turtlebot4, {"param_list": [], "skill": "init_robobt", "skill-life-cycle": "SUCCESS", "label": "init_robobt"}
"
'turtlebot4'
failure on parsing the line "18.62, [DEBUG], turtlebot2, {"param_list": [], "skill": "init_robobt", "skill-life-cycle": "SUCCESS", "label": "init_robobt"}
"
'turtlebot2'
failure on parsing the line "18.62, [DEBUG], turtlebot5, {"param_list": [], "skill": "init_robobt", "skill-life-cycle": "SUCCESS", "label": "init_robo

In [ ]:
cmc_df = pd.DataFrame.from_records(check_mc)
cte_df = pd.DataFrame.from_records(check_te)

In [ ]:
error_cmc_df = cmc_df[cmc_df.result == False]
error_cmc_df

In [ ]:
#cte_df_res = cte_df.groupby(["property", "result"])
cte_group_by_result = cte_df.groupby(["property", "result"])["result"].size().to_frame().rename(columns = {'result': 'count'}).reset_index()#.pivot(columns='result')
cte_group_by_result

In [ ]:
y = pd.pivot_table(cte_group_by_result,  values='count', index=['property'], columns=['result'] ).reset_index().rename(columns={True:'passed', False:'npassed'})
y

In [ ]:
y.assign(reason=lambda row:row.passed /(row.passed + row.npassed)*100 )

In [ ]:
cte_group_by_result


In [ ]:
error_cte_df = cte_df[cte_df.result == False].reset_index().sort_values(['exec_group', 'trial'])
with pd.option_context('display.max_rows', 120, 'display.max_columns', 10):
    display(error_cte_df)